In [5]:
import pandas as pd
import pickle
import numpy as np
import statsmodels.api as sm
import seaborn as sns
from haversine import haversine
import folium
from folium import plugins
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.stats import poisson
import statsmodels.api as sm
import matplotlib.pyplot as plt


df_base = pd.read_csv('../kaggle_data/input/train.csv')
# weather = pd.read_csv('../kaggle_data/input/weather.csv')
spray = pd.read_csv('../kaggle_data/input/spray.csv')

In [6]:
#Change everything to datetimes:
    
df_base.Date = pd.to_datetime(df_base.Date)
# weather.Date = pd.to_datetime(weather.Date)
spray.Date = pd.to_datetime(spray.Date)

In [ ]:
def recent_weather(date_range):
    recents = weather[weather.Date.isin(date_range)]
    results = {
        'Tmin': recents.Tmin.min(),
        'Tmin_avg' : recents.Tmin.mean(),
        'Hottest': recents.Heat.max(),
        'Coolest': recents.Cool.max()    
        }
    return results
twoweeks = pd.Timedelta('14 days')
get_past = lambda x: pd.date_range(x - twoweeks, x)
df_base['priorweeks'] = df_base.Date.apply(get_past)
df_base['recent_weather'] = df_base.priorweeks.apply(recent_weather)
df_base = df_base.dropna(how='any', axis=0)

In [ ]:
df_base['Tmin'] = df_base['recent_weather'].apply(lambda x: x['Tmin'])
df_base['Precip'] = pd.to_numeric(df_base['recent_weather'].apply(lambda x: x['Precip']), errors='coerce')
df_base['Hottest'] = pd.to_numeric(df_base['recent_weather'].apply(lambda x: x['Hottest']), errors='coerce')
df_base['Cool'] = pd.to_numeric(df_base['recent_weather'].apply(lambda x: x['Coolest']), errors='coerce')
df_base= df_base.dropna(how='any', axis=0)


df1 = df_base.groupby(['Latitude', 'Longitude','Species', 'Address', 'Date', 'Tmin', 'Hottest', 'Cool'], sort=True,
                 as_index = True)[['NumMosquitos', 'WnvPresent']].sum().reset_index()
df1 = df1.dropna(how='any', axis=0)


In [ ]:
from numpy import dot
from scipy.spatial.distance import cdist


cdist(df_base[['Latitude', 'Longitude']].values,spray[['Latitude', 'Longitude']].values, metric='euclidean')

In [ ]:
import numpy.linalg

is_near = lambda x, y: True if  numpy.linalg.norm(x-y)
    

In [ ]:
loc_array = np.asarray(spray[['Latitude', 'Longitude']])
pos_array = np.asarray(df_base[['Latitude','Longitude']])
pos_array[0]

In [ ]:
loc1 = loc_array-pos_array[0]
loc1

In [9]:
locations = sm.add_constant(spray[['Latitude', 'Longitude']], prepend=False)
locations.head()

,Latitude,Longitude,const
0,42.391623,-88.089163,1.0
1,42.391348,-88.089163,1.0
2,42.391022,-88.089157,1.0
3,42.390637,-88.089158,1.0
4,42.390410,-88.088858,1.0


In [28]:
spray.groupby(['Latitude', 'Longitude'])['Date'].apply(list).shape

(14294,)

In [25]:
df_base.groupby(['Address'],as_index=False)['Latitude', 'Longitude'].mean()

,Address,Latitude,Longitude
0,"1000 East 67th Street, Chicago, IL 60637, USA",41.773215,-87.600880
1,"1000 North Central Park Avenue, Chicago, IL 60...",41.899230,-87.716788
2,"1000 South California Avenue, Chicago, IL 6061...",41.869107,-87.696293
3,"1000 West 95th Street, Chicago, IL 60643, USA",41.721474,-87.648064
4,"1100 North Artesian Avenue, Chicago, IL 60622,...",41.903002,-87.688267
5,"1100 Roosevelt Road, Chicago, IL 60608, USA",41.867108,-87.654224
6,"1100 South California Avenue, Chicago, IL 6061...",41.868142,-87.696269
7,"1100 South Peoria Street, Chicago, IL 60608, USA",41.862292,-87.648860
8,"1100 South State Street, Chicago, IL 60605, USA",41.869216,-87.627561
9,"1100 West Chicago Avenue, Chicago, IL 60642, USA",41.896282,-87.655232


In [3]:
import folium
from folium import plugins

Chicago = folium.Map()


Chicago.add_child(plugins.HeatMap(locations.as_matrix(), radius=10, min_opacity = 0.5))
# Chicago.add_child(plugins.MarkerCluster(locations))
# Chicago.add_child(plugins.FastMarkerCluster(locations))
Chicago.fit_bounds([[41.644612, -87.930995], [42.01743, -87.531635]])
Chicago

In [12]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [10]:
import datetime
['Latitude','Longitude']

def close_points(trap_row, spray_row):
    return vincenty((spray_row['Latitude'],spray_row['Longitude']),(trap_row['Latitude'],trap_row['Longitude'])).miles


def close_spray(trap_row, spray_frame):
    end = trap_row['Date']
    start = end - datetime.timedelta('60 days')
    loc = trap_row['XY']
    temp_df = spray_frame[(if start < spray_frame['Date'] < end)]
    temp_df = temp_df[temp_df.apply(close)]
    